## Import libraries

In [ ]:
import gc
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold

## Prepare data for model training

In [ ]:
with open("../input/tps-june-data-preprocess/TPS_June_Dataset_Set1.txt", 'rb') as handle: 
    data = handle.read()

processed_data = pickle.loads(data)
train_df = processed_data['train_df']
test_df = processed_data['test_df']

Ytrain_oh = pd.get_dummies(train_df['target']).values

del processed_data
gc.collect()

In [ ]:
cat_cols = test_df.columns
train_df[cat_cols] = train_df[cat_cols].astype(int)
test_df[cat_cols] = test_df[cat_cols].astype(int)
cat_cols_indices = [train_df.columns.get_loc(col) for col in cat_cols]
print(cat_cols_indices)

In [ ]:
Xtrain = train_df.loc[:, train_df.columns != 'target'].copy()
Ytrain = train_df['target'].copy()
Ytrain_oh = pd.get_dummies(train_df['target']).copy()
Xtest = test_df.copy()

print("Xtrain: {} \nYtrain: {} \nYtrain_oh: {} \nXtest: {}".format(Xtrain.shape, Ytrain.shape, 
                                                                   Ytrain_oh.shape, Xtest.shape))

del train_df
del test_df
gc.collect()

## Build and validate the model

In [ ]:
FOLD = 10
NUM_SEED = 3

# Prediction Clipping Thresholds
p_min = 0.025
p_max = 1 - p_min

np.random.seed(3)
seeds = np.random.randint(0, 100, size=NUM_SEED)

oof_score = 0
y_pred_meta_cb = np.zeros((Ytrain.shape[0], 9))
y_pred_final_cb = np.zeros((Xtest.shape[0], 9))
counter = 0


for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain.values, Ytrain.values)):
        counter += 1

        train_x, train_y, train_y_oh = Xtrain.iloc[train], Ytrain.iloc[train], Ytrain_oh.iloc[train]
        val_x, val_y, val_y_oh = Xtrain.iloc[val], Ytrain.iloc[val], Ytrain_oh.iloc[val]

        model = CatBoostClassifier(
            objective='MultiClass',
            eval_metric='MultiClass',
            num_boost_round=5000,
            max_ctr_complexity=15,
            od_wait=1000, 
            od_type='Iter',
            use_best_model=True,
            bootstrap_type='Poisson',
            learning_rate=0.01465,
            reg_lambda=0.01864,
            subsample=0.68135,
            max_depth=6, 
            min_data_in_leaf=1,
            task_type='GPU',
            verbose=0
        )

        model.fit(train_x, train_y, eval_set=[(val_x, val_y)], 
                  cat_features=cat_cols_indices,
                  early_stopping_rounds=200, verbose=500)

        y_pred = model.predict_proba(val_x)
        #y_pred = np.clip(y_pred, p_min, p_max)
        y_pred_meta_cb[val] += y_pred
        y_pred_final_cb += model.predict_proba(Xtest)
        
        score = log_loss(val_y_oh, y_pred)
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_cb = y_pred_meta_cb / float(NUM_SEED)
y_pred_final_cb = y_pred_final_cb / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

In [ ]:
np.savez_compressed('./CB_Meta_Features.npz',
                    y_pred_meta_cb=y_pred_meta_cb, 
                    oof_score=oof_score,
                    y_pred_final_cb=y_pred_final_cb)

## Create submission file

In [ ]:
#y_pred_final_cb = np.clip(y_pred_final_cb, p_min, p_max)

test_df = pd.read_csv("../input/tabular-playground-series-jun-2021/test.csv")
submit_df = pd.DataFrame()
submit_df['id'] = test_df['id']
submit_df['Class_1'] = y_pred_final_cb[:,1]
submit_df['Class_2'] = y_pred_final_cb[:,2]
submit_df['Class_3'] = y_pred_final_cb[:,3]
submit_df['Class_4'] = y_pred_final_cb[:,4]
submit_df['Class_5'] = y_pred_final_cb[:,5]
submit_df['Class_6'] = y_pred_final_cb[:,6]
submit_df['Class_7'] = y_pred_final_cb[:,7]
submit_df['Class_8'] = y_pred_final_cb[:,8]
submit_df['Class_9'] = y_pred_final_cb[:,0]
submit_df.head()

In [ ]:
submit_df.to_csv("./CB_submission.csv", index=False)